<a href="https://colab.research.google.com/github/jgmsgabriel/NotebooksGeo/blob/main/Artigo_IA_Jun_2023_Cluster_Muzambinho_Satetelite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artigo - Inteligência Artificial

## Instalando e importnado bibliotecas

In [ ]:
!pip install geopandas geemap pyshp pycrs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=48eb3e19bc54a1f722716444a6c48846518941b3891e25977feee9a8eeae7797
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs


In [ ]:
import geemap
import geopandas
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-Zv3KVt43DsCrmO9B5AuaJ9YQnthenRldqr1AsEINss&tc=lr3P5LOnF7tkLf7s4xrM6GQu19un9oGH_dH6xSwr-w0&cc=cW8R8ax9oDjQvgaCfw8aM_Al9L5kgZi-OxRLq4jUUgw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVQPe39H4knTH9fOIOdfUNL0rVyQG1vP2AvqtWLWYSwY6D596EOebE

Successfully saved authorization token.


## Carregando Imagens e Shapefile

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
minas_shp = '/content/Mun_EPSG_4326.shp'
muz = geemap.shp_to_ee(minas_shp)
Map.addLayer(muz, {}, 'Muzambinho')

In [ ]:
# Região de interesse
roi = ee.Geometry.Polygon(
    [[[-406.62, -21.46], [-406.62, -21.26], [-406.42, -21.26], [-406.42, -21.46]]], None, False
)

# Minha Coleção de Imagem
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)).filterDate('2020-01-01','2021-01-01')
print('Quantas imagens eu tenho: ',collection.size().getInfo())

Quantas imagens eu tenho:  28


In [ ]:
# Selecionar a melhor imagem:
melhor_imagem = collection.sort('CLOUDY_PIXEL_PERCENTAGE', True).first()
print('ID: ',melhor_imagem.get('PRODUCT_ID').getInfo())
print('Porcentagem de nuvem: ', melhor_imagem.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())

ID:  S2A_MSIL2A_20200913T131251_N0214_R138_T23KLS_20200913T171631
Porcentagem de nuvem:  0.013493


In [ ]:
# Visualizar a imagem

# Aplicando escala - Tranformando em RGB
image = melhor_imagem.select(('B2', 'B3', 'B4', 'B8')).multiply(0.0001)

# AddLayer
Map.addLayer(image.clip(muz), {'bands':['B4', 'B3', 'B2'], 'min': 0.019, 'max': 0.19}, 'RGB Sentinel')
Map.addLayer(image.clip(muz), {'bands':['B8', 'B4', 'B3'], 'min': 0.019, 'max': 0.19}, 'False Color')
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
# Obtenha as dimensões em pixels
clipped_image = image.clip(muz)

# Obter informações sobre a imagem, como as propriedades da projeção e o número de pixels.
image_info = clipped_image.getInfo()
image_width = image_info['bands'][0]['dimensions'][0]   # Largura da imagem em pixels
image_height = image_info['bands'][0]['dimensions'][1]  # Altura da imagem em pixels

#print("Largura da imagem (em pixels):", image_width)
#print("Altura da imagem (em pixels):", image_height)

image_info

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]}]}

In [ ]:
# Obter informações sobre a imagem, como as propriedades da projeção e o número de pixels.
image_info_c = image.getInfo()
image_width_d = image_info_c['bands'][0]['dimensions'][0]   # Largura da imagem em pixels
image_height_d = image_info_c['bands'][0]['dimensions'][1]  # Altura da imagem em pixels

#print("Largura da imagem (em pixels):", image_width_d)
#print("Altura da imagem (em pixels):", image_height_d)
image_info_c

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 6.5535000000000005},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32723',
   'crs_transform': [10, 0, 300000, 0, -10, 7700020]}]}

In [ ]:
# Aplicando Indicie NDVI

# ((B8-B4)/(B8+B4))

ndvi = image.normalizedDifference(['B8','B4']).rename('NDVI')

Map.addLayer(ndvi.clip(muz), {'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen'], 'min':0, 'max':0.8}, 'NDVI')
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

## K-Means aplicadado RGB

In [ ]:
props = geemap.image_props(image)
props.getInfo()

{'NOMINAL_SCALE': 10, 'system:band_names': ['B2', 'B3', 'B4', 'B8']}

In [ ]:
training = image.sample(
    **{
        #'region': region,
        'region': muz,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training-rgb', False)
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer().clip(muz), {}, 'clusters-rgb')
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result.clip(muz), {'min': 1, 'max': 5, 'palette': legend_colors}, 'clusters-legend-rgb'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

## K-means aplicado ao NDVI

In [ ]:
props = geemap.image_props(ndvi)
props.getInfo()

{'NOMINAL_SCALE': 10, 'system:band_names': ['NDVI']}

In [ ]:
training = ndvi.sample(
    **{
        #'region': region,
        'region': muz,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training-ndvi', False)
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
# Cluster the input using the trained clusterer.
result_ndvi = ndvi.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result_ndvi.randomVisualizer().clip(muz), {}, 'clusters-ndvi')
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result_ndvi = result_ndvi.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result_ndvi.clip(muz), {'min': 1, 'max': 5, 'palette': legend_colors}, 'clusters-legend-ndvi'
)

Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

## Resultados

In [ ]:
Map

Map(bottom=583.0, center=[-18.646245142670608, -47.4609375], controls=(WidgetControl(options=['position', 'tra…

In [ ]:
left_layer = geemap.ee_tile_layer(result_ndvi.randomVisualizer().clip(muz), {}, 'clusters-ndvi')
right_layer = geemap.ee_tile_layer(result.randomVisualizer().clip(muz), {}, 'clusters-rgb')

In [ ]:
Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…